In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import jax
jax.config.update("jax_enable_x64", True)

In [2]:
import jax.numpy as jnp
from jax import jit, grad, jacobian, lax, vmap

import numpy as np
import matplotlib.pyplot as plt

In [3]:
import time
import scipy.optimize as so

In [4]:
n = 3
dt = 1e-1
numsteps = 100
rho = 1e6

h0 = np.load('workingh0.npz')['h0']

# h0 = jnp.array(np.diag(np.sort(np.pi*np.random.uniform(size=n))))
# mraw = np.random.normal(size=n**2).reshape((n,n)) + 1j*np.random.normal(size=n**2).reshape((n,n))
# m = jnp.array(0.5*(mraw + mraw.conj().T))
m = jnp.array([[1.0,1.5,-1.0],[1.5,1.0,0.0],[-1.0,0.0,1.0]])

print(jnp.mean(jnp.abs(h0 - h0.T)))
print(jnp.mean(jnp.abs(m - m.conj().T)))

0.0
0.0


In [5]:
# given initial condition and forcing f, return trajectory a
def propSchro(f, a0):
    a = [a0]
    for k in range(numsteps):
        thisham = h0 + f[k]*m
        evals, evecs = jnp.linalg.eigh(thisham)
        a.append( evecs @ jnp.diag(jnp.exp(-1j*dt*evals)) @ evecs.conj().T @ a[k] )
    
    a = jnp.stack(a, axis=0)
    return a

In [6]:
# given forcing f, IC a0, FC alpha, return cost
def cost(f, a0, alpha):
    a = propSchro(f, a0)
    resid = a[-1] - alpha
    pen = jnp.real(jnp.sum(resid * resid.conj()))
    return 0.5*jnp.sum(f**2) + 0.5*rho*pen

In [7]:
# (d/dx) \exp(-1j*dt*(h0 + x m))
# where you pass in the eigenvectors and eigenvalues of (h0 + x m)
def firstderiv(evecs, evals):
    amat = evecs.conj().T @ (-1j*dt*m) @ evecs
    dvec = -1j*dt*evals
    dvec1, dvec2 = jnp.meshgrid(dvec, dvec)
    mask = jnp.ones((n,n)) - jnp.eye(n)
    numer = jnp.exp(dvec1) - jnp.exp(dvec2)
    denom = (dvec1 - dvec2)*mask + jnp.eye(n)
    derivmat = mask*numer/denom + jnp.diag(jnp.exp(dvec))
    qmat = evecs @ (amat * derivmat) @ evecs.conj().T
    return qmat

In [8]:
mask = jnp.ones((n,n)) - jnp.eye(n)

# (d^2/dx^2) \exp(-1j*dt*(h0 + x m))
# where you pass in the eigenvectors and eigenvalues of (h0 + x m)
def secondderiv(evecs, rawevals):
    evals = (-1j*dt)*rawevals
    a = (evecs.conj().T @ m @ evecs) * (-1j*dt)
    expevals = jnp.exp(evals)
    evals1, evals2 = jnp.meshgrid(evals, evals, indexing='ij')
    mask = jnp.ones((n,n)) - jnp.eye(n)
    expevals1, expevals2 = jnp.meshgrid(expevals, expevals, indexing='ij')
    # first D_{ii}=D_{kk} term
    diagterm1 = expevals1*jnp.diag(jnp.diag(a*a))
    # second D_{ii}=D_{kk} term
    numer1 = -expevals1 + evals1*expevals1 - evals2*expevals1 + expevals2
    denom1 = (evals1-evals2)**2 + jnp.eye(n)
    maska = mask * a
    diagterm2 = np.eye(n) * 2*((numer1/denom1 * maska) @ maska)
    # first D_{ii}!=D_{kk} term
    frac1 = numer1/denom1 * mask
    term1 = frac1*2*(jnp.diag(a)*a).T
    # second D_{ii}!=D_{kk} term
    numer2 = -expevals1 + evals1*expevals2 - evals2*expevals2 + expevals2
    denom2 = (evals1-evals2)**2 + jnp.eye(n)
    frac2 = numer2/denom2 * mask
    term2 = frac2*2*(a*jnp.diag(a))
    # third D_{ii}!=D_{kk} term
    matij = mask*(1.0/((evals1-evals2) + jnp.eye(n)))
    matind1a = (expevals1 * matij) * a
    matind2a = (expevals2 * matij) * a
    term3 = 2*mask*((matind2a) @ (matij*a))
    term3 -= 2*matij*( matind1a @ maska )
    term3 -= 2*matij*( maska @ matind2a ) 
    # put it all together
    # udagru stands for "U^{\dagger} R U"
    udagru = term1 - term2 - term3 + diagterm1 + diagterm2
    return evecs @ udagru @ evecs.conj().T

In [9]:
def gradal(l,expderiv,a,matexp):
    ea = expderiv @ a
    gradvecs = [(l==0)*ea + (l>0)*jnp.zeros(n, dtype=jnp.complex128)]
    for k in range(1,numsteps):
        thisvec = (k<l)*jnp.zeros(n, dtype=jnp.complex128)
        thisvec += (k==l)*ea
        thisvec += (k>l)*(matexp[k] @ gradvecs[k-1])
        gradvecs.append( thisvec )
    
    # zeroblock = jnp.zeros((l, n), dtype=jnp.complex128)
    return jnp.stack(gradvecs, axis=0)

In [10]:
def onehessrow(lamb,mu,ed,a,ga):
    part1 = jnp.real(mu @ ed @ a)
    part2 = jnp.real(lamb.T @ ed @ ga.T)
    return part1 + part2

In [11]:
manyeigh = vmap(jnp.linalg.eigh)
vfd = vmap(firstderiv, in_axes=(0,0))
vsd = vmap(secondderiv, in_axes=(0,0))
vgradal = vmap(gradal, in_axes=(0,0,0,None))
vohr = vmap(onehessrow)

In [12]:
# adjoint method
def adjgrad(f, a0, alpha):
    manyhams = jnp.expand_dims(h0,0) + jnp.expand_dims(f,(1,2))*jnp.expand_dims(m,0)
    allevals, allevecs = manyeigh(manyhams)
    expevals = jnp.exp(-1j*dt*allevals)
    a = [a0]
    matexp = []
    for k in range(numsteps):
        matexp.append( allevecs[k] @ jnp.diag(expevals[k]) @ allevecs[k].conj().T )
        a.append( matexp[k] @ a[k] )
   
    # forward trajectory
    a = jnp.stack(a, axis=0)
    
    # compute cost
    resid = a[-1] - alpha
    pen = jnp.real(jnp.sum(resid * resid.conj()))
    thecost = 0.5*jnp.sum(f**2) + 0.5*rho*pen

    # initialize lambda
    # we are storing "lambda conjugate" throughout this calculation
    alllamb = [rho*resid.conj()]
    for k in range(numsteps-1,-1,-1):
        kk = (numsteps-1) - k
        alllamb.append( alllamb[kk] @ matexp[k] )
    
    # backward trajectory
    alllamb = jnp.flipud(jnp.stack(alllamb, axis=0))
    
    # first critical calculation
    allexpderivs = vfd(allevecs, allevals)
    
    # output gradient we want
    ourgrad = jnp.einsum('ai,aij,aj->a',alllamb[1:],allexpderivs,a[:-1])
    thegrad = f + jnp.real(ourgrad)
    
    return thecost, thegrad

In [13]:
# second-order adjoint method
def adjhess(f, a0, alpha):
    manyhams = jnp.expand_dims(h0,0) + jnp.expand_dims(f,(1,2))*jnp.expand_dims(m,0)
    allevals, allevecs = manyeigh(manyhams)
    expevals = jnp.exp(-1j*dt*allevals)
    a = [a0]
    matexp = []
    for k in range(numsteps):
        matexp.append( allevecs[k] @ jnp.diag(expevals[k]) @ allevecs[k].conj().T )
        a.append( matexp[k] @ a[k] )
    
    # forward trajectory
    a = jnp.stack(a, axis=0)
    
    # compute cost
    resid = a[-1] - alpha
    pen = jnp.real(jnp.sum(resid * resid.conj()))
    thecost = 0.5*jnp.sum(f**2) + 0.5*rho*pen

    # initialize lambda
    # we are storing "lambda conjugate" throughout this calculation
    alllamb = [rho*resid.conj()]
    for k in range(numsteps-1,-1,-1):
        kk = (numsteps-1) - k
        alllamb.append( alllamb[kk] @ matexp[k] )
    
    # backward trajectory
    alllamb = jnp.flipud(jnp.stack(alllamb, axis=0))
    
    # first critical calculation
    allexpderivs = vfd(allevecs, allevals)
    
    # compute gradient
    ourgrad = jnp.einsum('ai,aij,aj->a',alllamb[1:],allexpderivs,a[:-1])
    thegrad = f + jnp.real(ourgrad)

    # compute gradient of a w.r.t. f
    # grada tensor stores the gradient of the n-dimensional vector a[k] with respect to f[l]
    lvec = jnp.arange(numsteps,dtype=jnp.int16)
    grada = vgradal(lvec, allexpderivs, a[:-1], matexp)
    grada = jnp.transpose(grada,(1,0,2))
     
    # create and propagate mu
    # as before, let us store and propagate "mu conjugate"
    allmu = [rho*grada[numsteps-1,:,:].conj()]
    for k in range(numsteps-1,-1,-1):
        kk = (numsteps-1) - k
        prevmu1 = allmu[kk] @ matexp[k]
        zb1 = jnp.zeros((k,n),dtype=jnp.complex128)
        zb2 = jnp.zeros((numsteps-k-1,n),dtype=jnp.complex128)
        prevmu2 = jnp.concatenate([zb1,jnp.expand_dims(alllamb[k+1].T @ allexpderivs[k],0),zb2])
        allmu.append(prevmu1+prevmu2)

    # backward trajectory
    allmu = jnp.flipud(jnp.stack(allmu, axis=0))
        
    # second critical calculation
    allexpderivs2 = vsd(allevecs, allevals)
    
    # compute Hessian
    gradapad = jnp.concatenate([jnp.zeros((1,numsteps,n),dtype=jnp.complex128), grada[:-1,:,:]])
    parts12 = vohr(alllamb[1:],allmu[1:],allexpderivs,a[:-1],gradapad)
    part3 = jnp.diag(jnp.real(jnp.einsum('ai,aij,aj->a',alllamb[1:],allexpderivs2,a[:-1])))
    thehess = jnp.eye(numsteps) + parts12 + part3
    
    return thecost, thegrad, thehess

In [14]:
jcost = jit(cost)
jadjgrad = jit(adjgrad)
jadjhess = jit(adjhess)

In [15]:
mya0 = jnp.eye(n)[0]
myalpha = jnp.array([jnp.sqrt(0.0),0,jnp.sqrt(1.0)])
myalpha /= jnp.linalg.norm(myalpha)

In [17]:
ftest = jnp.array(np.random.normal(size=numsteps))
mycost = jcost(ftest, mya0, myalpha)
mycost2, mygrad = jadjgrad(ftest, mya0, myalpha)
mycost3, mygrad2, myhess = jadjhess(ftest, mya0, myalpha)

In [24]:
def hessobj(x):
    jx = jnp.array(x)
    jc, jg, jh = jadjhess(jx,mya0,myalpha)
    return jc.item(), np.array(jg), np.array(jh)

In [25]:
def gradobj(x):
    jx = jnp.array(x)
    jc, jg = jadjgrad(jx,mya0,myalpha)
    return jc.item(), np.array(jg)

In [30]:
opt = fides.Optimizer(
    hessobj, ub=np.inf*np.ones(numsteps), lb=-np.inf*np.ones(numsteps)
)

In [ ]:
opt_f, opt_x, opt_grad, opt_hess = opt.minimize(np.array(ftest))

2025-02-11 14:40:00 fides(INFO)  iter|    fval   |   fdiff  | tr ratio |tr radius|  ||g||  | ||step||| step|acc
2025-02-11 14:40:00 fides(INFO)     0| +7.74E+05 |    NaN   |    NaN   | 1.0E+00 | 7.4E+05 |   NaN   | NaN |1
2025-02-11 14:40:02 fides(INFO)     1| +3.68E+05 | -4.1E+05 | +8.0E-01 | 1.0E+00 | 7.4E+05 | 1.0E+00 |  2d |1
2025-02-11 14:40:03 fides(INFO)     2| +1.19E+05 | -2.5E+05 | +4.9E-01 | 2.0E+00 | 4.8E+05 | 2.0E+00 |  2d |1
2025-02-11 14:40:04 fides(INFO)     3| +1.19E+05 | +1.6E+05 | -3.9E-01 | 2.0E+00 | 2.7E+05 | 2.0E+00 |  2d |0
2025-02-11 14:40:05 fides(INFO)     4| +5.68E+04 | -6.3E+04 | +9.4E-01 | 5.0E-01 | 2.7E+05 | 5.0E-01 |  2d |1
2025-02-11 14:40:05 fides(INFO)     5| +5.17E+04 | -5.2E+03 | +7.7E-02 | 1.0E+00 | 1.6E+05 | 1.0E+00 |  2d |1
2025-02-11 14:40:05 fides(INFO)     6| +2.11E+04 | -3.1E+04 | +9.2E-01 | 2.5E-01 | 3.0E+05 | 2.5E-01 |  2d |1
2025-02-11 14:40:06 fides(INFO)     7| +1.10E+04 | -1.0E+04 | +6.8E-01 | 5.0E-01 | 1.1E+05 | 5.0E-01 |  2d |1
2025-02-

In [32]:
opt_f

36.94636147612717